# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 27 2022 9:01AM,242832,12,HH-36094,Hush Hush,Released
1,Jul 27 2022 9:01AM,242832,12,HH-36095,Hush Hush,Released
2,Jul 27 2022 9:01AM,242832,12,HH-36096,Hush Hush,Released
3,Jul 27 2022 9:01AM,242832,12,HH-36097,Hush Hush,Released
4,Jul 27 2022 9:01AM,242832,12,HH-36098,Hush Hush,Released
5,Jul 27 2022 9:01AM,242832,12,HH-36099,Hush Hush,Released
6,Jul 27 2022 9:01AM,242832,12,HH-36100,Hush Hush,Released
7,Jul 27 2022 9:01AM,242832,12,HH-36101,Hush Hush,Released
8,Jul 27 2022 9:01AM,242832,12,HH-36102,Hush Hush,Released
9,Jul 27 2022 9:01AM,242832,12,HH-36103,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,242827,Released,67
36,242828,Released,1
37,242830,Released,28
38,242831,Released,1
39,242832,Released,23


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242827,NaN,NaN,67.0
242828,NaN,NaN,1.0
242830,NaN,NaN,28.0
242831,NaN,NaN,1.0
242832,NaN,NaN,23.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242723,4.0,1.0,26.0
242733,1.0,0.0,1.0
242735,12.0,0.0,3.0
242748,1.0,39.0,0.0
242781,2.0,3.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242723,4,1,26
242733,1,0,1
242735,12,0,3
242748,1,39,0
242781,2,3,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242723,4,1,26
1,242733,1,0,1
2,242735,12,0,3
3,242748,1,39,0
4,242781,2,3,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242723,4,1,26
1,242733,1,,1
2,242735,12,,3
3,242748,1,39,
4,242781,2,3,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 27 2022 9:01AM,242832,12,Hush Hush
23,Jul 27 2022 8:31AM,242830,10,ISDIN Corporation
51,Jul 27 2022 8:30AM,242792,10,ISDIN Corporation
69,Jul 27 2022 8:29AM,242831,19,"GCH Granules USA, Inc."
70,Jul 27 2022 8:29AM,242827,10,ISDIN Corporation
137,Jul 27 2022 8:24AM,242828,10,Cosmo International Fragrances
138,Jul 27 2022 8:17AM,242826,10,Beach Products Inc
156,Jul 27 2022 8:16AM,242825,10,ISDIN Corporation
174,Jul 27 2022 8:16AM,242824,10,Beach Patient Assistance
175,Jul 27 2022 8:13AM,242823,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 27 2022 9:01AM,242832,12,Hush Hush,,,23
1,Jul 27 2022 8:31AM,242830,10,ISDIN Corporation,,,28
2,Jul 27 2022 8:30AM,242792,10,ISDIN Corporation,,,18
3,Jul 27 2022 8:29AM,242831,19,"GCH Granules USA, Inc.",,,1
4,Jul 27 2022 8:29AM,242827,10,ISDIN Corporation,,,67
5,Jul 27 2022 8:24AM,242828,10,Cosmo International Fragrances,,,1
6,Jul 27 2022 8:17AM,242826,10,Beach Products Inc,,,18
7,Jul 27 2022 8:16AM,242825,10,ISDIN Corporation,,,18
8,Jul 27 2022 8:16AM,242824,10,Beach Patient Assistance,,,1
9,Jul 27 2022 8:13AM,242823,10,ISDIN Corporation,,2,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 9:01AM,242832,12,Hush Hush,23,,
1,Jul 27 2022 8:31AM,242830,10,ISDIN Corporation,28,,
2,Jul 27 2022 8:30AM,242792,10,ISDIN Corporation,18,,
3,Jul 27 2022 8:29AM,242831,19,"GCH Granules USA, Inc.",1,,
4,Jul 27 2022 8:29AM,242827,10,ISDIN Corporation,67,,
5,Jul 27 2022 8:24AM,242828,10,Cosmo International Fragrances,1,,
6,Jul 27 2022 8:17AM,242826,10,Beach Products Inc,18,,
7,Jul 27 2022 8:16AM,242825,10,ISDIN Corporation,18,,
8,Jul 27 2022 8:16AM,242824,10,Beach Patient Assistance,1,,
9,Jul 27 2022 8:13AM,242823,10,ISDIN Corporation,7,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 9:01AM,242832,12,Hush Hush,23,,
1,Jul 27 2022 8:31AM,242830,10,ISDIN Corporation,28,,
2,Jul 27 2022 8:30AM,242792,10,ISDIN Corporation,18,,
3,Jul 27 2022 8:29AM,242831,19,"GCH Granules USA, Inc.",1,,
4,Jul 27 2022 8:29AM,242827,10,ISDIN Corporation,67,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 9:01AM,242832,12,Hush Hush,23.0,NaN,NaN
1,Jul 27 2022 8:31AM,242830,10,ISDIN Corporation,28.0,NaN,NaN
2,Jul 27 2022 8:30AM,242792,10,ISDIN Corporation,18.0,NaN,NaN
3,Jul 27 2022 8:29AM,242831,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
4,Jul 27 2022 8:29AM,242827,10,ISDIN Corporation,67.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 27 2022 9:01AM,242832,12,Hush Hush,23.0,0.0,0.0
1,Jul 27 2022 8:31AM,242830,10,ISDIN Corporation,28.0,0.0,0.0
2,Jul 27 2022 8:30AM,242792,10,ISDIN Corporation,18.0,0.0,0.0
3,Jul 27 2022 8:29AM,242831,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
4,Jul 27 2022 8:29AM,242827,10,ISDIN Corporation,67.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2670933,170.0,2.0,12.0
12,242832,23.0,0.0,0.0
15,242733,1.0,0.0,1.0
16,485611,0.0,4.0,0.0
19,2185217,26.0,4.0,0.0
20,971071,60.0,46.0,7.0
21,485633,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2670933,170.0,2.0,12.0
1,12,242832,23.0,0.0,0.0
2,15,242733,1.0,0.0,1.0
3,16,485611,0.0,4.0,0.0
4,19,2185217,26.0,4.0,0.0
5,20,971071,60.0,46.0,7.0
6,21,485633,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,170.0,2.0,12.0
1,12,23.0,0.0,0.0
2,15,1.0,0.0,1.0
3,16,0.0,4.0,0.0
4,19,26.0,4.0,0.0
5,20,60.0,46.0,7.0
6,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,170.0
1,12,Released,23.0
2,15,Released,1.0
3,16,Released,0.0
4,19,Released,26.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,12.0,0.0,1.0,0.0,0.0,7.0,0.0
Executing,2.0,0.0,0.0,4.0,4.0,46.0,0.0
Released,170.0,23.0,1.0,0.0,26.0,60.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,12.0,0.0,1.0,0.0,0.0,7.0,0.0
1,Executing,2.0,0.0,0.0,4.0,4.0,46.0,0.0
2,Released,170.0,23.0,1.0,0.0,26.0,60.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,12.0,0.0,1.0,0.0,0.0,7.0,0.0
1,Executing,2.0,0.0,0.0,4.0,4.0,46.0,0.0
2,Released,170.0,23.0,1.0,0.0,26.0,60.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()